In [21]:
#!/usr/bin/env python

from Bio import Entrez
from Bio import Medline

MAX_COUNT = 10
TERM = 'Entyvio'

print('Getting {0} publications regarding {1}...'.format(MAX_COUNT, TERM))
Entrez.email = 'A.N.Other@example.com'
h = Entrez.esearch(db='pubmed', retmax=MAX_COUNT, term=TERM)
result = Entrez.read(h)
print('Total number of publications containing {0}: {1}'.format(TERM, result['Count']))
ids = result['IdList']
h = Entrez.efetch(db='pubmed', id=ids, rettype='medline', retmode='text')
records = Medline.parse(h)

titles = []
sources = []
for record in records:
    ti = record.get('SO')
    titles.append(ti)
print('titles: {0}'.format(titles))

Getting 10 publications regarding Entyvio...
Total number of publications containing Entyvio: 1837
titles: ['Aliment Pharmacol Ther. 2023 Jul;58(2):262-263. doi: 10.1111/apt.17610.', 'Crohns Colitis 360. 2023 May 29;5(3):otad031. doi: 10.1093/crocol/otad031. eCollection 2023 Jul.', 'J Immunother Cancer. 2023 Jun;11(6):e007195. doi: 10.1136/jitc-2023-007195.', 'Medicine (Baltimore). 2023 May 19;102(20):e33784. doi: 10.1097/MD.0000000000033784.', 'Aliment Pharmacol Ther. 2023 Jun 15. doi: 10.1111/apt.17616.', 'Paediatr Drugs. 2023 Jun 15. doi: 10.1007/s40272-023-00579-6.', 'Sci Transl Med. 2023 Jun 14;15(700):eabq4006. doi: 10.1126/scitranslmed.abq4006. Epub 2023 Jun 14.', 'Rev Esp Enferm Dig. 2023 Jun 14. doi: 10.17235/reed.2023.9491/2023.', 'Zhonghua Yi Xue Za Zhi. 2023 Jun 20;103(23):1759-1766. doi: 10.3760/cma.j.cn112137-20221216-02663.', 'J Med Cases. 2023 May;14(5):155-161. doi: 10.14740/jmc4041. Epub 2023 May 31.']


In [7]:
print(h)

<_io.TextIOWrapper encoding='UTF-8'>


In [3]:
import json
import requests
import xml.etree.ElementTree as ET

def lambda_handler(event, context):
    
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    api_url = base_url + "esearch.fcgi"
    params = {
        "db": "pubmed",
        "retmode": "xml",
        "retmax": 5,  # Number of papers to retrieve (adjust as needed)
        "term": event['search'],  # Modify the search query to match your needs
        "sort": "pub+date"
    }
    
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        xml_content = response.content
        root = ET.fromstring(xml_content)
        pubmed_ids = [id_node.text for id_node in root.findall(".//Id")]
        
        url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={','.join(pubmed_ids)}&rettype=xml"
        doi_response = requests.get(url)
        records = doi_response.content
    
        # Parse the XML records
        root = ET.fromstring(records)
    
        # List to store dictionaries
        pubmed_list = []
    
        # Find the relevant elements and extract their text
        for pubmed_article in root.findall(".//PubmedArticle"):
            # Dictionary to store the information
            pubmed_dict = {}
    
            # Extract title
            title_element = pubmed_article.find(".//ArticleTitle")
            pubmed_dict["Title"] = title_element.text if title_element is not None else "N/A"
    
            # Extract detailed date or fallback to year
            pub_date_element = pubmed_article.find(".//ArticleDate")
            if pub_date_element is not None:
                year_element = pub_date_element.find("Year")
                year = year_element.text if year_element is not None else "N/A"
    
                month_element = pub_date_element.find("Month")
                month = month_element.text if month_element is not None else "N/A"
    
                day_element = pub_date_element.find("Day")
                day = day_element.text if day_element is not None else "N/A"
    
                # Combine the detailed date components
                pubmed_dict["Date"] = f"{day}/{month}/{year}"
            else:
                # Fallback to year if detailed date is not available
                year_element = pubmed_article.find(".//PubDate/Year")
                pubmed_dict["Date"] = year_element.text if year_element is not None else "N/A"
    
            # Extract DOI
            doi_element = pubmed_article.find(".//ArticleId[@IdType='doi']")
            pubmed_dict["DOI"] = doi_element.text if doi_element is not None else "N/A"
    
            # Append the dictionary to the list
            pubmed_list.append(pubmed_dict)

        # Create the JSON response
        json_response = {
            'statusCode': 200,
            'body': {}
        }
        
        paper_objects = {}
        for i, paper in enumerate(pubmed_list):
            paper_objects[f"Paper {i+1}"] = paper
        
        json_response['body']['papers'] = paper_objects

        return json_response

    else:
        return {
            'statusCode': 500
        }


In [5]:
event = {'search': "Entyvio"}
lambda_handler(event)

{'statusCode': 200,
 'body': {'papers': {'Paper 1': {'Title': 'Real-world Evidence Comparing Tofacitinib and Vedolizumab in Anti-TNF-experienced Patients With Ulcerative Colitis.',
    'Date': '26/06/2023',
    'DOI': '10.1093/ibd/izad115'},
   'Paper 2': {'Title': 'Effectiveness of Non-Budesonide Therapies in Management of Microscopic Colitis: A Systematic Review and Meta-analysis.',
    'Date': '26/06/2023',
    'DOI': '10.1007/s40265-023-01914-4'},
   'Paper 3': {'Title': 'Letter: Experience of switching vedolizumab treatment from intravenous to subcutaneous formulation in Hungary.',
    'Date': '2023',
    'DOI': '10.1111/apt.17610'},
   'Paper 4': {'Title': 'Comparing Patient-Reported Outcomes Among Anti-TNF Experienced Patients With Ulcerative Colitis Initiating Vedolizumab Versus Tofacitinib.',
    'Date': '29/05/2023',
    'DOI': '10.1093/crocol/otad031'},
   'Paper 5': {'Title': 'Concurrent immune checkpoint inhibition and selective immunosuppressive therapy in patients with i